In [15]:
from dataloader import GcMTLDataset
import pandas as pd
import torch

label_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/gc/n-labels/all_label.csv'
df = pd.read_csv(label_path, header=None)
keep_same_psize = 1000
file_name = df.iloc[:, 0].values
file_label = df.iloc[:, 1].values
root = '/data/wsi/TCTGC2625-features/plip-fine'
num_classes = [5]
num_task = 0
fine_concat = True
dataset = GcMTLDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat, keep_same_psize=keep_same_psize)
# dataset = GcMTLFineDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True)
for i, (f, l, t, f) in enumerate(data_loader):
    print(l.shape)
    print(t)

torch.Size([9])
tensor([1, 1, 0, 1, 2, 2, 2, 2, 2])


In [18]:
import torch
import pandas as pd
import shutil, os
from tqdm import tqdm

fn = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df = pd.read_csv(fn)
label_wsi_names = df['wsi_name'].tolist()
label_wsi_names = [i.replace('.pt', '') for i in label_wsi_names]
print(len(label_wsi_names))
wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume1'
output_wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume-deprecated13'
wsi_names = os.listdir(wsi_path)
diff = set(wsi_names) - set(label_wsi_names)
diff = list(diff)
len(diff)

9854


0

In [17]:
# for i in diff:
#     shutil.move(os.path.join(wsi_path, i), os.path.join(output_wsi_path, i))

In [20]:
from modules.vit_wsi.model_v1 import Model_V1

model = Model_V1(
        in_dims=512,
        num_classes=[1,5,3],
        depth=4,
        num_heads=4,
        proj_drop=0.02,
        attn_drop=0.02,
        drop_path=0.01,)
sum_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(sum_params / 1024 ** 2)

43.09864139556885


In [16]:
import torch
from model import MIL_MTL

def count_parameters(model):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params /= 1024 ** 2
    return params

num_task = 3
num_classes=[1,5,3]
input_dim = 256
model2 = MIL_MTL(num_classes=num_classes, num_task=num_task, input_dim=input_dim)
print((count_parameters(model2)))

size = (5, 10, 256) # [B, N, C]
mask = torch.ones(size[0], size[1], size[2])
a = torch.randn(size)
task_id = torch.randint(0, num_task, (size[0],))
output = model2(a, task_id)
print(output.shape)
# print(output.shape)

0.19251346588134766
torch.Size([5, 9])


In [15]:
from utils import multi_class_scores_mtl
import numpy as np

n = 100
n_classes = 9
bag_labels = np.random.randint(0, n_classes, n)
class_labels = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL', 'AGC', 'BV', 'M', 'T']
bag_logits = np.random.rand(*(n, n_classes))
wsi_names = ['WSI' + str(i) for i in range(n)]
threshold = 0.5

multi_class_scores_mtl(bag_labels, bag_logits, class_labels, wsi_names, threshold, eval_only=False)
# def multi_class_scores_mtl(bag_labels, bag_logits, class_labels, wsi_names, threshold, eval_only):

[ERROR] WSI23 risk prediction is wrong: [0.9903, 0.4942, 0.865, 0.8805, 0.106, 0.3009]
[ERROR] WSI46 risk prediction is wrong: [0.9718, 0.5357, 0.4098, 0.0345, 0.1743, 0.6252]
[ERROR] WSI55 risk prediction is wrong: [0.9905, 0.8902, 0.9189, 0.1448, 0.4494, 0.5315]
[INFO] confusion matrix for cancer labels:
5 5 6
Confusion Matrix for 61 data
+-----------+------+--------+------+-------+------+-----+------+
| 实际\预测 | NILM | ASC-US | LSIL | ASC-H | HSIL | AGC | 总计 |
+-----------+------+--------+------+-------+------+-----+------+
|    NILM   |  1   |   1    |  3   |   2   |  1   |  1  |  9   |
|   ASC-US  |  0   |   1    |  2   |   7   |  3   |  3  |  16  |
|    LSIL   |  0   |   2    |  4   |   1   |  3   |  1  |  11  |
|   ASC-H   |  0   |   2    |  0   |   2   |  1   |  1  |  6   |
|    HSIL   |  1   |   1    |  4   |   1   |  1   |  1  |  9   |
|    AGC    |  0   |   0    |  4   |   3   |  3   |  0  |  10  |
|    总计   |  2   |   7    |  17  |   16  |  12  |  7  |  61  |
+-----------+--

([0.5527777777777778,
  0.5545454545454546,
  0.4696969696969697,
  0.3974358974358974,
  0.4411764705882353,
  0.5679012345679013,
  0.638888888888889,
  0.4629629629629629],
 0.22,
 array([0.0625    , 0.36363636, 0.33333333, 0.11111111, 0.        ,
        0.16666667, 0.53333333, 0.25      ]),
 array([0.14285714, 0.23529412, 0.125     , 0.08333333, 0.        ,
        0.2       , 0.5       , 0.23076923]),
 array([0.08695652, 0.28571429, 0.18181818, 0.0952381 , 0.        ,
        0.18181818, 0.51612903, 0.24      ]))

In [13]:
Model_V1(
        in_dims=args.input_dim,
        num_classes=args.num_classes,
        depth=4,
        num_heads=4,
        proj_drop=0.02,
        attn_drop=0.02,
        drop_path=0.01,)

([torch.Size([1000, 25, 256]), torch.Size([1000, 25, 256])],
 [torch.Size([1000, 25]), torch.Size([1000, 25])])

In [11]:
masks[0][900]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])